In [1]:
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split    
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from RSData import *
from HaiYangData import *
from mpl_toolkits.basemap import Basemap
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sea_ice_model import * 
from sklearn import svm
import graphviz
import pandas as pd
import numpy as np
import glob,os
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches




In [2]:
VV_HH = 'VV'

train_dir = r'd:\python_workfile\sea_ice_classification\\training6'

# csv_dir = train_dir + '\\csv\\\split_VV_HH\\'+VV_HH +'\\-10_-20threshold'
csv_dir = r'D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio'
csv_files = glob.glob(csv_dir + '\*.csv')

In [4]:
save_dir = r'D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic'

In [13]:
satellite = r'AARI'
sensor = r'AARI'
hy_sca = HaiYangData(satellite=satellite, sensor=sensor,resolution=30000)
# 将WGS 84坐标（4326）转化为极射投影
crs = CRS.from_epsg(4326)
crs = CRS.from_string("epsg:4326")
crs = CRS.from_proj4("+proj=latlon")
crs = CRS.from_user_input(4326)
crs2 = CRS(proj="aeqd")
transformer = HaiYangData.set_transformer(crs,crs2)
transformer_back = HaiYangData.set_transformer(crs2,crs)


In [18]:
FI = mpatches.Patch(color='maroon', label='Fast Ice')
OI = mpatches.Patch(color='orange', label='Old Ice')
FYI = mpatches.Patch(color='lime', label='First Year Ice')
YI = mpatches.Patch(color='dodgerblue', label='Young Ice')
N = mpatches.Patch(color='midnightblue', label='Nilas')


In [26]:
for val_csv_file in csv_files[:]:
    day = val_csv_file.split('\\')[-1].split('.')[0]
    val_data = pd.read_csv(val_csv_file,index_col=None,encoding='ANSI')
#     val_data['week'] = val_data['time'].apply(strftime_week)
#     val_data['day'] = val_data['time'].apply(strftime_day)
#     val_data['julian_day'] = val_data['time'].apply(strftime_julian_day)
#     val_data['julian_week'] = val_data['time'].apply(strftime_julian_week)
#     nor_dataframe(val_data)
    process_sea_ice_train_dataframe(val_data,fyi_myi=False)
    value_array = np.empty(shape=(val_data.shape[0],6))


    value_array[:,0] = val_data['lat']
    value_array[:,1] = val_data['lon']
    value_array[:,2],value_array[:,3] = transformer.transform(value_array[:,0], value_array[:,1])
    value_array[:,4] = val_data['ice_type']

    grid_array = np.zeros((hy_sca.nlat, hy_sca.nlon))

    x = (value_array[:,2] / hy_sca.resolution).astype(np.int16)
    y = (value_array[:,3] / hy_sca.resolution).astype(np.int16)


    grid_array[y,x] = value_array[:,4]

    x_map, y_map = hy_sca.get_map_grid(transformer_back)
    
    
    grid_array[grid_array == 0] = np.nan

    
    
    
    
    # 绘制真值
    fig = plt.figure(figsize=(9, 9))
    fig.add_subplot(111)
    fig.set_tight_layout(True) # reduce the spaces from margin outside the axis 

    hy_m = Basemap(projection='npaeqd', boundinglat=66, lon_0=90., resolution='c')
    hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)

    hy_m.fillcontinents()
    hy_m.drawmapboundary()
    hy_m.drawparallels(np.arange(-90., 120., 10.), labels=[1, 0, 0, 0])
    hy_m.drawmeridians(np.arange(-180., 180., 60.), labels=[0, 0, 0, 1])
    plt.title('True mask '+str(day))
    plt.legend(loc='upper right',handles=[FI,OI ,FYI, YI ,N], title='Ice Type')
    plt.savefig(save_dir+'\\True '+str(day)+'.png')
    plt.close()
    
    print(f'now processed csv file {val_csv_file}')
    print(save_dir+'\\True '+str(day)+'.png')
    
    

C:\Users\Administrator\AppData\Local\Temp/ipykernel_4248/2832217777.py:41: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)


now processed csv file D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio\20190514.csv
D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic\True 20190514.png


C:\Users\Administrator\AppData\Local\Temp/ipykernel_4248/2832217777.py:41: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)


now processed csv file D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio\20190515.csv
D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic\True 20190515.png


C:\Users\Administrator\AppData\Local\Temp/ipykernel_4248/2832217777.py:41: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)


...

now processed csv file D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio\20210411.csv
D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic\True 20210411.png


C:\Users\Administrator\AppData\Local\Temp/ipykernel_4248/2832217777.py:41: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)


now processed csv file D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio\20210412.csv
D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic\True 20210412.png


C:\Users\Administrator\AppData\Local\Temp/ipykernel_4248/2832217777.py:41: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  hy_m.pcolor(x_map, y_map, data=grid_array, cmap=plt.cm.jet,shading = 'auto',vmax=5,vmin=1,latlon = True)


now processed csv file D:\python_workfile\sea_ice_classification\training6\csv\polarization_ratio\20210413.csv
D:\python_workfile\sea_ice_classification\data\mask\aari\proj\pic\True 20210413.png
